In [6]:
from lmnav.emb_transfer.old_eai_policy_2_2 import EAIPolicy

device = 'cuda'
torch.set_grad_enabled(False)

teacher_ckpt = "/coc/testnvme/pputta7/.experiments/embodiment-transfer/202304171546-jzPl/out/new_checkpoints/ckpt.300.pth"
teacher_state_dict = torch.load(teacher_ckpt)

In [7]:
def batch_to(device):
    batch['rgb'] = batch['rgb'].to(device)
    batch['depth'] = batch['depth'].to(device)
    batch['imagegoal'] = batch['imagegoal'].to(device)
    
test_recurrent_hidden_states = torch.zeros((1, 2, 512), device=device)
prev_actions = torch.zeros(1, 1, device=device, dtype=torch.long)
not_done_masks = torch.zeros(1, 1, device=device, dtype=torch.bool)
rgb_frames = []

In [ ]:
observations = env.reset()

In [ ]:
step = 0
done = False

# while step < 500 and not done:
# batch data up
batch = batch_obs([observations])
batch = apply_obs_transforms_batch(batch, obs_transforms)
batch_to('cuda')

rgb_frames.append(observations['rgb'].copy())

# query policy for next action
policy_result = teacher.act(batch, test_recurrent_hidden_states, prev_actions, not_done_masks, deterministic=False)
next_action = policy_result.actions.item()
print(next_action)
test_recurrent_hidden_states = policy_result.rnn_hidden_states
prev_actions[0][0] = next_action

# pass data through environment
observations = env.step(next_action)
infos = env.get_metrics()
step += 1
done = infos['success'] > 0

In [ ]:
features, hx, _ = teacher.net(batch, test_recurrent_hidden_states, prev_actions, not_done_masks)
action_dist = teacher.action_distribution(features)
action_dist.logits

In [ ]:
import cv2

imgs = rgb_frames

# Get the list of images
# Create the video writer
video_writer = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 24, (imgs[0].shape[1], imgs[0].shape[0]))

# Write the images to the video writer
for image in imgs:
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    video_writer.write(image)

# Close the video writer
video_writer.release()